In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-3.5-turbo"

In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

model = ChatOpenAI(api_key=OPENAI_API_KEY, model=MODEL)
embeddings = OpenAIEmbeddings()

respuesta = model.invoke("Contame un chiste").content
respuesta


'¿Por qué los pájaros no usan Facebook?\n\nPorque ya tienen Twitter.'

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Proyecto 044-23 Arraigada Russovich V2.pdf")
pages = loader.load_and_split()


In [5]:
from langchain.prompts import PromptTemplate

template = """
Responde la pregunta basandote en el contexto proporcionado debajo. Si no puedes responder la pregunta, contesta "No lo sé".

Contexto: {context}

Pregunta: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="aca hay un contexto!", question="esta es una pregunta.")

'\nResponde la pregunta basandote en el contexto proporcionado debajo. Si no puedes responder la pregunta, contesta "No lo sé".\n\nContexto: aca hay un contexto!\n\nPregunta: esta es una pregunta.\n'

In [6]:
chain = prompt | model

In [7]:
chain.invoke(
    {
        "context":"La mejor startup de Argentina es Efibud.",
        "question":"Cual es la mejor empresa emprendedora en argentina?"
    }
)

AIMessage(content='La mejor empresa emprendedora en Argentina es Efibud.', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 67, 'total_tokens': 81}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b843f713-f77b-4d64-867d-05108021a172-0')

In [8]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [12]:
retriever = vectorstore.as_retriever()
retriever.invoke("Como está la capacidad de la empresa? Estan ajustados?")

[Document(page_content='“Estudio de la capacidad y mejoras de las operaciones en una empresa de catering”  \nAutores: Javier I. Arraigada – Matko I. Russovich         Página 36 de 57 las instalaciones, cualquier incremento en la demanda (ya sea por nuevos contratos con \ncomedores o incrementos de ventas de viandas) podría requerir una expansión o una \noptimización de procesos para evitar déficits.  \n \nEn cuanto a los eventos, se puede concluir que los recursos disponibles superan  ampliamente a \nlos necesarios, lo cual indica una capacidad ociosa considerable. Este excedente ofrece una \noportunidad estratégica para que la empresa explore nuevos mercados o para intentar aumentar \nel volumen de negocios en eventos sin necesidad de inver sión adicional en capacidad. Sin \nembargo, también representa un desperdicio considerable de recursos y, por lo tanto, una \noportunidad de ahorro.', metadata={'source': 'Proyecto 044-23 Arraigada Russovich V2.pdf', 'page': 39}),
 Document(page_co

In [19]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever, # Trae el contexto usando la pregunta que le pasamos.
        "question": itemgetter("question")
    }
    | prompt
    | model
)

respuesta = chain.invoke({"question":"Que se concluye del analisis de la demanda?"})
print(respuesta.content)

Se concluye que el análisis de la demanda permitió obtener una estimación de la capacidad requerida, lo que proporcionó una visión integral de las tendencias y patrones estacionales que influyen en el comportamiento de la demanda. Además, se logró una comprensión más profunda de los factores que la impulsan, lo que ha sido fundamental para mejorar la capacidad de anticipar y satisfacer las necesidades del mercado de manera eficiente y rentable.
